In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
! cd drive/MyDrive/Interview/NN\ Models

In [13]:
import os
os.chdir('drive/MyDrive/Interview/NN Models')

In [14]:
import pandas as pd
data = pd.read_csv('dataset.csv')
data.head()

,image,mask,helmet
0,D:/Work/Neo4j/Regan - Full Time AI Engineer - ...,yes,yes
1,D:/Work/Neo4j/Regan - Full Time AI Engineer - ...,no,yes
2,D:/Work/Neo4j/Regan - Full Time AI Engineer - ...,yes,no
3,D:/Work/Neo4j/Regan - Full Time AI Engineer - ...,no,yes
4,D:/Work/Neo4j/Regan - Full Time AI Engineer - ...,invisible,no


In [15]:
data['image'][0]

'D:/Work/Neo4j/Regan - Full Time AI Engineer - Technical test/interview/task2/final/0/0.jpg'

In [19]:
data['image'] = data['image'].replace('D:/Work/Neo4j/Regan - Full Time AI Engineer - Technical test/interview/task2/', '')

In [23]:
import re
data['image'] = pd.Series([re.sub(r'D:/Work/Neo4j/Regan - Full Time AI Engineer - Technical test/interview/task2/', '', item) for item in data['image']])

In [24]:
data.head()

,image,mask,helmet
0,final/0/0.jpg,yes,yes
1,final/0/1.jpg,no,yes
2,final/1/0.jpg,yes,no
3,final/2/0.jpg,no,yes
4,final/2/1.jpg,invisible,no


In [25]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [26]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split = 0.2)

train_data = datagen.flow_from_dataframe(
    dataframe=data,
    directory = './',
    x_col = 'image',
    y_col="mask",
    target_size=(64, 64),
    batch_size = 32,
    shuffle = True,
    class_mode = "categorical",
    subset = 'training'
)

valid_data = datagen.flow_from_dataframe(
    dataframe = data,
    directory = './',
    x_col = 'image',
    y_col = "mask",
    target_size = (64, 64),
    batch_size =32,
    shuffle = True,
    class_mode = "categorical",
    subset = 'validation'
)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 1075 validated image filenames belonging to 4 classes.
Found 268 validated image filenames belonging to 4 classes.


In [27]:
import tensorflow as tf
from tensorflow.compat.v1.layers import MaxPooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import InputLayer
# from keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [28]:
vgg16_model = tensorflow.keras.applications.vgg16.VGG16()

553467904/553467096 [==============================] - 3s 0us/step


In [29]:
model_mask = Sequential()
model_mask.add( InputLayer(input_shape=(64, 64, 3), name='image_input') )

In [30]:
for layer in vgg16_model.layers[1:-4]:
  model_mask.add(layer)

In [31]:
for layer in model_mask.layers:
  layer.trainable = False

In [32]:
model_mask.add(Conv2D(64, (2, 2), padding='same'))
model_mask.add(Activation('relu'))
model_mask.add(MaxPool2D(pool_size=(2, 2), strides = (1,1), padding='same'))

model_mask.add(Conv2D(64, (2, 2), padding='same'))
model_mask.add(Activation('relu'))
model_mask.add(MaxPool2D(pool_size=(2, 2), strides = (1,1), padding='same'))

In [33]:
model_mask.add(Flatten())

In [34]:
model_mask.add(Dense(128))
model_mask.add(Activation('relu'))
model_mask.add(Dense(64))
model_mask.add(Activation('relu'))
model_mask.add(Dropout(0.25))
model_mask.add(Dense(4))
model_mask.add(Activation('softmax'))

In [35]:
from keras import metrics
model_mask.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(
                  learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD'),
              metrics=[metrics.categorical_accuracy])

In [36]:
model_mask.fit(
        train_data,
        epochs=100,
        validation_data=valid_data,
        )

Epoch 1/100
34/34 [==============================] - 508s 14s/step - loss: 1.3039 - categorical_accuracy: 0.3165 - val_loss: 1.1734 - val_categorical_accuracy: 0.3396
Epoch 2/100
34/34 [==============================] - 3s 90ms/step - loss: 1.1301 - categorical_accuracy: 0.3979 - val_loss: 1.1619 - val_categorical_accuracy: 0.3060
Epoch 3/100
34/34 [==============================] - 3s 92ms/step - loss: 1.1196 - categorical_accuracy: 0.3919 - val_loss: 1.1314 - val_categorical_accuracy: 0.3321
Epoch 4/100
34/34 [==============================] - 3s 92ms/step - loss: 1.0904 - categorical_accuracy: 0.4568 - val_loss: 1.1206 - val_categorical_accuracy: 0.3321
Epoch 5/100
34/34 [==============================] - 3s 93ms/step - loss: 1.0431 - categorical_accuracy: 0.4461 - val_loss: 1.0859 - val_categorical_accuracy: 0.3507
Epoch 6/100
34/34 [==============================] - 3s 95ms/step - loss: 1.0240 - categorical_accuracy: 0.4901 - val_loss: 1.1193 - val_categorical_accuracy: 0.3433
Epo

In [37]:
labels_mask = (train_data.class_indices)
labels_mask = dict((v,k) for k,v in labels_mask.items())
labels_mask

{0: 'invisible', 1: 'no', 2: 'wrong', 3: 'yes'}

In [38]:
model_mask.save_weights("model_mask.h5")

In [39]:
model_mask_json = model_mask.to_json()
with open("model_mask.json", "w") as json_file:
    json_file.write(model_mask_json)

Helmet Model

In [41]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [42]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split = 0.2)

train_data = datagen.flow_from_dataframe(
    dataframe=data,
    directory = './',
    x_col = 'image',
    y_col="helmet",
    target_size=(64, 64),
    batch_size = 32,
    shuffle = True,
    class_mode = "binary",
    subset = 'training'
)

valid_data = datagen.flow_from_dataframe(
    dataframe = data,
    directory = './',
    x_col = 'image',
    y_col = "helmet",
    target_size = (64, 64),
    batch_size =32,
    shuffle = True,
    class_mode = "binary",
    subset = 'validation'
)

Found 1075 validated image filenames belonging to 2 classes.
Found 268 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [43]:
model_helmet = Sequential()
model_helmet.add( InputLayer(input_shape=(64, 64, 3), name='image_input') )

In [44]:
for layer in vgg16_model.layers[1:-4]:
  model_helmet.add(layer)

In [45]:
for layer in model_helmet.layers:
  layer.trainable = False

In [46]:
model_helmet.add(Conv2D(64, (2, 2), padding='same'))
model_helmet.add(Activation('relu'))
model_helmet.add(MaxPool2D(pool_size=(2, 2), strides = (1,1), padding='same'))

model_helmet.add(Conv2D(64, (2, 2), padding='same'))
model_helmet.add(Activation('relu'))
model_helmet.add(MaxPool2D(pool_size=(2, 2), strides = (1,1), padding='same'))

In [47]:
model_helmet.add(Flatten())
model_helmet.add(Dense(128))
model_helmet.add(Activation('relu'))
model_helmet.add(Dense(64))
model_helmet.add(Activation('relu'))
model_helmet.add(Dropout(0.25))
model_helmet.add(Dense(1))
model_helmet.add(Activation('sigmoid'))

In [48]:
from keras import metrics
model_helmet.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.SGD(
                  learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD'),
              metrics=[metrics.binary_accuracy])

In [49]:
model_helmet.fit(
        train_data,
        epochs=100,
        validation_data=valid_data,
        )

Epoch 1/100
34/34 [==============================] - 4s 103ms/step - loss: 0.6804 - binary_accuracy: 0.5481 - val_loss: 0.6166 - val_binary_accuracy: 0.6828
Epoch 2/100
34/34 [==============================] - 3s 95ms/step - loss: 0.6137 - binary_accuracy: 0.6887 - val_loss: 0.6004 - val_binary_accuracy: 0.6828
Epoch 3/100
34/34 [==============================] - 3s 92ms/step - loss: 0.6038 - binary_accuracy: 0.7038 - val_loss: 0.5873 - val_binary_accuracy: 0.6828
Epoch 4/100
34/34 [==============================] - 3s 91ms/step - loss: 0.6047 - binary_accuracy: 0.6773 - val_loss: 0.5775 - val_binary_accuracy: 0.6828
Epoch 5/100
34/34 [==============================] - 3s 93ms/step - loss: 0.5914 - binary_accuracy: 0.6891 - val_loss: 0.5707 - val_binary_accuracy: 0.6828
Epoch 6/100
34/34 [==============================] - 3s 92ms/step - loss: 0.5692 - binary_accuracy: 0.6990 - val_loss: 0.5408 - val_binary_accuracy: 0.6940
Epoch 7/100
34/34 [==============================] - 3s 91ms/st

In [50]:
labels_helmet = (train_data.class_indices)
labels_helmet = dict((v,k) for k,v in labels_helmet.items())
labels_helmet

{0: 'no', 1: 'yes'}

In [51]:
model_helmet.save_weights("model_helmet.h5")

In [52]:
model_helmet_json = model_helmet.to_json()
with open("model_helmet.json", "w") as json_file:
    json_file.write(model_helmet_json)